In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from lstm import lstm_model, train_lstm
from data_processing import get_and_save_data, get_train_test_data, convert_to_1d
from globals import DATA
from training_functions import *



In [ ]:
get_and_save_data(dir_path=DATA)

model, train_loader, encoder_input_train, decoder_output_train, encoder_input_test, decoder_output_test = get_data_and_model(input_dim=2, hidden_dim=128, layer_dim=3, output_dim=1, epochs = 5)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_curve = train_lstm(model, device, train_loader, num_epochs=10, lr = 0.001)

In [ ]:
plt.plot(loss_curve)

In [ ]:
view_train_test_result(model, encoder_input_train, decoder_output_train, encoder_input_test, decoder_output_test)